In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from importlib import reload
from matplotlib import rc
from neurobiases import EMSolver, TriangularModel
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
K = 1

In [ ]:
# Create triangular model and draw data
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10,
    N=10,
    K=K,
    corr_cluster=0.05,
    corr_back=0.,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0,
    coupling_scale=0.25,
    coupling_rng=2332,
    tuning_distribution='gaussian',
    tuning_sparsity=0.5,
    tuning_loc=0,
    tuning_scale=0.25,
    tuning_rng=23456542,
    stim_distribution='uniform')
X, Y, y = tm.generate_samples(n_samples=2000, rng=2332)

# No transform, only bounds

In [ ]:
# Run sparse solver
solver = EMSolver(
    X, Y, y, K=K,
    solver='scipy_lbfgs',
    max_iter=1000,
    tol=1e-7,
    penalize_B=False,
    rng=948512,
    fit_intercept=False,
    Psi_transform=None,
    initialization='random')

In [ ]:
n_deltas = 21
deltas_init = np.linspace(-1., 1., n_deltas)

for idx, delta in enumerate(deltas_init):
    copy = solver.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr[0])

In [ ]:
init_copy = solver.copy()
solver.fit_em()
fit_copy = solver.copy()

In [ ]:
init_params = np.zeros((n_deltas, solver.get_params().size))

for idx, delta in enumerate(deltas_init):
    print(idx)
    copy = init_copy.copy()
    copy.identifiability_transform(delta)
    init_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
deltas_fit = np.linspace(-0.17, 0.17, n_deltas)

for idx, delta in enumerate(deltas_fit):
    copy = solver.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr[0])

In [ ]:
fit_ident_params = np.zeros((n_deltas, solver.get_params().size))

for idx, delta in enumerate(deltas_fit):
    copy = fit_copy.copy()
    copy.identifiability_transform(delta)
    fit_ident_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
fit_params = np.zeros((n_deltas, solver.get_params().size))

for idx, delta in enumerate(deltas_init):
    print(idx)
    copy = init_copy.copy()
    copy.identifiability_transform(delta)
    copy.fit_em()
    fit_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
pca = PCA(2)
init_transformed = pca.fit_transform(init_params)
fit_params_init_transformed = pca.transform(fit_params)
fit_params_ident_init_transformed = pca.transform(fit_ident_params)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(init_transformed[:, 0], init_transformed[:, 1], marker='o', color='black')
ax.plot(fit_params_init_transformed[:, 0], fit_params_init_transformed[:, 1], color='red')
ax.plot(fit_params_ident_init_transformed[:, 0], fit_params_ident_init_transformed[:, 1],
        color='green')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(fit_params_init_transformed[:, 0], fit_params_init_transformed[:, 1], color='red')
#ax.plot(fit_params_ident_init_transformed[:, 0], fit_params_ident_init_transformed[:, 1],
#        color='green')

In [ ]:
pca = PCA(2)
fit_ident_transformed = pca.fit_transform(fit_ident_params)
fit_transformed = pca.transform(fit_params)

In [ ]:
plt.plot(pca.components_[0][:10])

In [ ]:
plt.plot(fit_transformed[:, 0], deltas_init)

In [ ]:
pca.components_[0, 10 + 10 + 10 * 10 + 11]

In [ ]:
for i in range(10):
    plt.plot(fit_transformed[:, 0], fit_params[:, i])
    
plt.xlabel('PCA projection')
plt.ylabel('coupling parameter fit value')

In [ ]:
plt.plot(deltas_init, fit_params[:, 8])

In [ ]:
plt.plot(fit_transformed[:, 0], fit_params[:, 10 + 10 + 10 * 10], marker='o')

In [ ]:
plt.plot(pca.components_[0])

In [ ]:
pca.components_.shape

In [ ]:
plt.plot(fit_transformed[:, 0], (fit_params[:, 10:20] * pca.components_[0][10:20]).sum(axis=1), marker='o')

In [ ]:
plt.plot(deltas_init, fit_params[:, 1])

In [ ]:
plt.plot(pca.components_[0][10:20])

In [ ]:
plt.plot(pca.components_[1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(fit_ident_transformed[:, 0], fit_ident_transformed[:, 1], marker='o')
ax.plot(fit_transformed[:, 0], fit_transformed[:, 1], marker='x')

In [ ]:
plt.plot(deltas_init, fit_params[:, 7])

In [ ]:
pca = PCA(2)
fit_transformed = pca.fit_transform(init_params)
fit_ident_transformed = pca.transform(fit_params)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(fit_ident_transformed[:, 0], fit_ident_transformed[:, 1], marker='o')
#ax.plot(fit_transformed[:, 0], fit_transformed[:, 1], marker='x')